# Naive Bayes 

In [190]:
import numpy as np
np.set_printoptions(precision=6)

## Bayes theorem
$$\underbrace{P(B|A)}_2 = \underbrace{\frac{P(A|B)}{P(A)}}_3 \underbrace{P(B)}_1$$
<ol>
<li>The prior is what you believed about $B$ before having encountered a new and relevant piece of information (i.e., $A$). In our case, this is the probability that a given document belongs to class $B$(China) and class $\overline{B}$(not China).</li>

<li>The posterior is what you believe (or ought to, if you are rational) about $B$ after having encountered a new and relevant piece of information. With more and more training examples, this should get close to the ground truth. In our case, this means that given information $A$(training data), what is the probability that the it came from class $B$ </li>

<li>The quotient of the likelihood divided by the marginal probability of the new piece of information indexes the informativeness of the new information for your beliefs about $B$.  </li> 

Bayes theorem can be simplified into
$${P(B|A)} \propto {{P(A|B)}}{P(B)}$$
This is because the marginal probability $P(A)$ is a normalizing constant, i.e., it's present in calculating posterior of all classes and all it does is give posterior as probabilities (sum to 1). We don’t care what the true posterior value is (might be needed when comparing different models), we just want to know which class has the higher posterior value. Furthermore, computing the marginal probability isn't trivial in most datasets. 

## Aim

The training set below can be interprted as words in a document and the labels represent the class of the document which can be China or not China. <br>
<b>Training Set</b> <br>
<table class="tg">
  <tr>
    <th class="tg-yw4l">DocID</th>
    <th class="tg-yw4l">Words in Doc<br></th>
    <th class="tg-yw4l">Doc class = China?<br></th>
  </tr>
  <tr>
    <td class="tg-yw4l">1</td>
    <td class="tg-yw4l">Chinese Beijing Chinese<br></td>
    <td class="tg-yw4l">1<br></td>
  </tr>
  <tr>
    <td class="tg-yw4l">2</td>
    <td class="tg-yw4l">Chinese Chinese Shanghai<br></td>
    <td class="tg-yw4l">1<br></td>
  </tr>
  <tr>
    <td class="tg-yw4l">3</td>
    <td class="tg-yw4l">Chinese Macao<br></td>
    <td class="tg-yw4l">1<br></td>
  </tr>
  <tr>
    <td class="tg-yw4l">4</td>
    <td class="tg-yw4l">Tokyo Japan Chinese<br></td>
    <td class="tg-yw4l">0<br></td>
  </tr>
</table>

An array represent a document and its index a word and its value represent the count of appearance. 

In [7]:
X = np.array([
    [2,1,0,0,0,0],
    [2,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,0,0,0,1,1]
])
y = np.array([1,1,1,0])

Our goal is to find the best class for the document. The best class in NB classification is the most likely or maximum a posteriori ( MAP ) class which is simply MLE(Maximum Likelihood Estimate) multiplied by a prior, where MLE is the relative frequency and corresponds to the most likely value of each parameter given the training data. For instance the MLE of the prior for class $C$ in this example is ${\frac{C}{C+\overline{C}}}$.
<br><br>
$${c}_{MAP} = \text{argmax}_{c \in C} \; \; [ \log \hat P(c) + \sum_i  \log \hat{P}(t_i|c)]$$
Where $t_i$s are the unique terms in the document. They are essentially the features of our model. 
One immediate question is why logs and why are they summed? The answer is that in Eq(2), there maybe many features and many conditional probabilities are multiplied. This can result in a floating point underflow. It is therefore better to perform the computation by adding logarithms of probabilities instead of multiplying probabilities. The class with the highest log probability score is still the most probable; $\log (x+y) = \log (x) + \log (y))$ and the logarithm function is monotonic.

In [72]:
class MultinomailNB(object):
    def __init__(self, alpha = 1):
        self.alpha = alpha #See [1]
    def fit(self, X, y):
        #Grouping training data by class
        separated = [[x for x,t in zip(X,y) if t == c] for c in np.unique(y)]
        print ('Separated Data: ',separated)
        count_sample = X.shape[0]
        self.class_log_prior = [np.log(len(x)/count_sample) for x in separated]
        print ('Priors: ', self.class_log_prior)
        count = np.array([np.sum(x, axis=0)+self.alpha for x in separated])
        print ('Counts of appearances: ',count)
        #log probabilities of each word
        self.feature_log_prob = np.log(count/count.sum(axis=1)[np.newaxis].T)
        print ('Probabilities of each term: ', self.feature_log_prob)
    def predict_log_proba(self, X):
        return [(self.feature_log_prob*x).sum(axis=1)+self.class_log_prior for x in X]
    def predict(self, X):
        return [np.argmax(x) for x in self.predict_log_proba(X)]

[1] We use the alpha as a smoothing factor; to fix a problem with the MLE estimate whereby probability is zero for a term-class combination that did not occur in the training data.  If the term Mandarin in the training data only occurred in China documents, then the MLE estimates for not China, will be zero if Mandarin appeared in a test sample with a class not China. The zero probability for Mandarin cannot be "conditioned away" no matter how strong the evidence for the class not China from other features. The estimate is 0 because of sparseness: the training data are never large enough to represent the frequency of rare events adequately.

## Train

In [67]:
model = MultinomailNB()
model.fit(X,y)

Separated Data:  [[array([1, 0, 0, 0, 1, 1])], [array([2, 1, 0, 0, 0, 0]), array([2, 0, 1, 0, 0, 0]), array([1, 0, 0, 1, 0, 0])]]
Priors:  [-1.3862943611198906, -0.2876820724517809]
Counts of appearances:  [[2 1 1 1 2 2]
 [6 2 2 2 1 1]]
Probabilities of each term:  [[-1.504077 -2.197225 -2.197225 -2.197225 -1.504077 -1.504077]
 [-0.847298 -1.94591  -1.94591  -1.94591  -2.639057 -2.639057]]


## Test

Data
<table class="tg">
  <tr>
    <th class="tg-yw4l">docID</th>
    <th class="tg-yw4l">Words in Document<br></th>
  </tr>
  <tr>
    <td class="tg-yw4l">1</td>
    <td class="tg-yw4l">Chinese Chinese Chinese Tokyo Japan</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2</td>
    <td class="tg-yw4l">Beijing Shanghai Tokyo Japan</td>
  </tr>
</table>

In [68]:
X_test = np.array([[3,0,0,0,1,1],[0,1,1,0,1,1]])

In [75]:
model.predict(X_test)

array([1, 0])

## Guassian Naive Bayes
When the data has continuous variables, then the above model is not suitable. Fitting the model to a gaussian distribution is a good solution for working with such data. Note that not every data follows a Gaussian distribution, but it's a pretty good assumption.

In [383]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score

### Data
HTRU2 pulsar candidate [dataset](http://archive.ics.uci.edu/ml/datasets/HTRU2#)

In [500]:
path = 'data/HTRU2/HTRU_2.csv'

In [501]:
full = pd.read_csv(path, header=None)
X = full.drop(8, 1)
X = X.as_matrix()

In [502]:
y = full.as_matrix(columns=[8])
class_names=[0,1]
#Number of true labels
print ('{}% of the data is true labels'.format(np.count_nonzero(y)/len(full)*100))

9.157447759526203% of the data is true labels


In [503]:
#Normalize data
scaler = StandardScaler()
train = scaler.fit_transform(X)

In [504]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, stratify=y)

In [528]:
class GaussianNB(object):
    def __init__(self):
        pass

    def fit(self, X, y):
        self.separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.model = np.array([np.c_[np.mean(x, axis=0), np.std(x, axis=0)] for 
                              x in self.separated])
        count_sample = X.shape[0]
        self.class_log_prior = [np.log(len(x)/count_sample) for x in self.separated]
        return self
    def gaussian(self, x, mean, std):
        return np.log((np.exp(-1*(x-mean)**2/(2*std**2)))/(std*np.sqrt(2*np.pi)))
    
    def predict_log_proba(self, X):
        results = []
        for x in X:
            example = []
            for c in self.model:
                prob = 0
                for e, i in zip(x, c):
                    prob += self.gaussian(e, *i)
                example.append(prob)
            results.append(example)
        return np.asarray(results)
    def predict(self, X):
        return [np.argmax(x) for x in (self.predict_log_proba(X)+self.class_log_prior)] #Watch out for the prior

In [506]:
def accmetrics(labels, preds, averaging):
    prec = precision_score(labels, preds, average=averaging)*100
    rand = metrics.adjusted_rand_score(labels.squeeze(), preds.squeeze())*100
    return rand, prec 

In [507]:
nb = GaussianNB().fit(X_train, y_train)
preds = nb.predict(X_test)

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


In [508]:
accmetrics(y_test, np.asarray(preds), 'binary')

(64.174310857987322, 62.110726643598611)

The results are not great but the task wasn't easy either. We can confirm that it works on a much simpler classification task.

In [521]:
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
nb = GaussianNB().fit(X_train, y_train)
preds = nb.predict(X_test)

In [526]:
accmetrics(y_test, np.asarray(preds), 'micro')

(85.881730049476928, 94.73684210526315)

Another thing to note is the importance of meaninful priors. In nebula detection, adding a naive constant prior improved the accuracy by ~4 points whereas it adversely affected the iris dataset. Perhaps a more meaningful choice given the limited dataset is a uniform distribution. 

### References
<ol><li>[Kenzo Takahashi's blog post](http://kenzotakahashi.github.io/naive-bayes-from-scratch-in-python.html)</li>
<li>[Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html)
